In [1]:
%cd ../../

/mnt/sda/otani/Experiments/eval-detection


In [2]:
from mim.utils import DEFAULT_CACHE_DIR
import os
import mmcv
from mmcv.utils.config import Config
import numpy as np
import pandas as pd

from src.extensions.dataset.coco_custom import CocoOtcDataset
from tqdm import tqdm
import matplotlib.pyplot as plt
from shutil import copyfile

from concurrent.futures import ProcessPoolExecutor, as_completed
%matplotlib inline

In [3]:
out_pkl_a = "outputs/otc_search_param_candidates/20211027_003051/RetinaNet.pkl"
out_pkl_b = "outputs/otc_search_param_candidates/20211027_003051/YOLOF.pkl"
model_cfg = "retinanet_r50_fpn_2x_coco"
cfg = cfg = Config.fromfile(os.path.join(DEFAULT_CACHE_DIR, model_cfg + ".py"))
data_cfg = cfg.data

In [4]:
def eval_otc(results, params):
    ot_costs = coco.eval_OTC(results, **params, get_average=False)
    key = f"alpha={params['alpha']:.1}_beata={params['beta']:.1f}"
    return key, ot_costs

In [5]:
results_a = mmcv.load(out_pkl_a)
results_b = mmcv.load(out_pkl_b)
out_dir = os.path.dirname(out_pkl_a)

coco = CocoOtcDataset(
    "data/coco/annotations/instances_val2017.json",
    data_cfg.test.pipeline,
    test_mode=True,
)

file_names = [x['file_name'] for x in coco.data_infos]
df_a = {"image": file_names}
df_b = {"image": file_names}

params = {"alpha": 0.5, "beta": 0.4}
step = 0.2
for results, df in zip((results_a, results_b), (df_a, df_b)):
    progress = tqdm(len(np.arange(0.1, 0.99, step))**2)
    with ProcessPoolExecutor(10) as pool:
        futures = []
        for p in np.arange(0.1, 0.99, step):
            for q in np.arange(0.1, 0.99, step):
                params = {"alpha": p, "beta": q}

                future = pool.submit(eval_otc, results, params)
                futures.append(future)

        for future in as_completed(futures):
            key, ot_costs = future.result()
            df[key] = ot_costs
            progress.update(1)

df_a = pd.DataFrame(df_a)
df_b = pd.DataFrame(df_b)
cols = df_a.columns.tolist()
cols.sort()
df_a = df_a[cols]
df_b = df_b[cols]

loading annotations into memory...
Done (t=0.57s)
creating index...
index created!


0it [00:00, ?it/s]

In [ ]:
eps = 0.01
is_b_better = df_a.values[:, :-1] > df_b.values[:, :-1] + eps
n = is_b_better.shape[-1]
n_b_better = is_b_better.sum(axis=-1)
candidates= np.where((n_b_better > n/2-5) & (n_b_better < n/2+5))[0]
sampled_images = df_a.iloc[candidates]["image"]
print(len(sampled_images))

698


In [ ]:
src_base = "outputs/run_evaluation/20211027_045631/"
out_base = "data/processed/user_study/"
for model in ["RetinaNet", "YOLOF"]:
    out_dir = os.path.join(out_base, model)
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
    
    for fn in sampled_images:
        copyfile(f"{src_base}{model}/{fn}", f"{os.path.join(out_dir, fn)}")
